## 1. 환경설정

In [21]:
import json

from dotenv import load_dotenv
from torch.onnx.symbolic_opset11 import chunk
from tornado.escape import json_decode

load_dotenv('../.env')

True

## 2. 다양한 문서 형식 처리하기
### 2.1 PDF 문설

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("./data/transformer.pdf")
pdf_docs = pdf_loader.load()
print(len(pdf_docs))
print(pdf_docs)
print(pdf_docs[0].metadata)
print(pdf_docs[0].page_content)


## 2.2 웹 문서

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])
web_docs = web_loader.load()
print(len(web_docs))
web_docs[0].metadata
print(web_docs[0].page_content)

## 2.3 JSON 파일

In [42]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",  # messages 배열의 content 필드만 추출
    text_content=True  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수: ", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수:  5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [43]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",  # messages 배열의 모든 아이템을 추출
    text_content=False,  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\uc548\ub155\ud558\uc138\uc694 \uc5ec\ub7ec\ubd84, \uc624\ub298 \ud68c\uc758 \uc2dc\uac04 \ud655\uc778\ucc28 \uc5f0\ub77d\ub4dc\ub9bd\ub2c8\ub2e4."}


In [44]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있음)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs:
    decoded_data = json.loads(doc.page_content)

    # 딕셔너리 형태로 저장
    # decoded_json_docs.append({
    #     "metadata": doc.metadata,
    #     "page_content": json.dumps(decoded_data, ensure_ascii=False)
    # })

    # Langchain의 Document 객체
    document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
# print("처음 문서의 메타데이터: \n", decoded_json_docs[0]["metadata"])
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata)
print("-" * 50)
# print("처음 문서의 내용: \n", decoded_json_docs[0]["page_content"])
print("처음 문서의 내용: \n", decoded_json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."}


In [45]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata


json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",
    content_key="content",
    metadata_func=metadata_func,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.json', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [46]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.jsonl', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [47]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata


json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    metadata_func=metadata_func,
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': '/Users/ken/IdeaProjects/vtopia/langchain_study/conda_rag/data/kakao_chat.jsonl', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


## 2.4 CSV 파일

In [49]:
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

문서의 수: 10
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': './data/kbo_teams_2023.csv', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


## 3. 효과적인 텍스트 분할 전략
### 3.1 RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 이름에서 알 수 있듯이 재귀적으로 텍스트를 분할합니다.
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 나눕니다.
- 일반적으로 CharacterTextSplitter보다 더 엄격하게 크기를 준수하려고 합니다.

In [50]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 청크 크기
    chunk_overlap=200,  # 청크 중 중복되는 부분 크기
    length_function=len,  # 글자 수를 기준을 분할
    separators=["\n\n", "\n"]  # 구분자
)
texts = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

생성된 텍스트 청크 수: 52
각 청크의 길이: [981, 910, 975, 451, 932, 998, 904, 907, 995, 385, 926, 953, 216, 920, 996, 829, 975, 910, 906, 870, 929, 961, 945, 997, 195, 977, 968, 947, 933, 965, 938, 915, 733, 952, 945, 948, 618, 980, 989, 994, 624, 945, 914, 946, 918, 988, 929, 929, 849, 812, 814, 817]


In [51]:
# 각 청크의 시작 부분과 끝 부분 확인
for text in texts[:5]:
    print(text.page_content[:200])
    print("-" * 200)
    print(text.page_content[-200:])
    print("=" * 200)
    print()

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions

mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine transla
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 3.2 정규표현식 사용

In [52]:
# 문장을 구분하여 분할(마침표, 느낌표, 물음표 다음에 공백이 온느 경우 문장의 끝으로 판단)
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0,
    separator=r'(?<=[.!?])\s+',
    is_separator_regex=True,
    keep_separator=True
)

texts = text_splitter.split_documents(json_docs)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인
for text in texts[:5]:
    print(text.page_content[:50])
    print("-" * 50)
    print(text.page_content[-50:])
    print("=" * 50)
    print()

Created a chunk of size 11, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10


생성된 텍스트 청크 수: 9
각 청크의 길이: [31, 9, 15, 7, 11, 11, 27, 13, 13]

안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.
--------------------------------------------------
안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.

네, 안녕하세요.
--------------------------------------------------
네, 안녕하세요.

오후 2시에 하기로 했어요.
--------------------------------------------------
오후 2시에 하기로 했어요.

확인했습니다.
--------------------------------------------------
확인했습니다.

회의실은 어디인가요?
--------------------------------------------------
회의실은 어디인가요?



## 3.3 토큰 수를 기반으로 분할
### (1) tiktoken
- OpenAi에서 만든 BPE Tokenizer

In [55]:
len(pdf_docs[0].page_content)

2853

In [56]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=300,
    chunk_overlap=0,
)

chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(chunk.page_content[:50])
    print("-" * 50)
    print(chunk.page_content[-50:])
    print("=" * 50)
    print()

생성된 청크 수: 3
각 청크의 길이: [1140, 1374, 337]
Provided proper attribution is provided, Google he
--------------------------------------------------
ng more parallelizable and requiring significantly

less time to train. Our model achieves 28.4 BLEU o
--------------------------------------------------
countless long days designing various parts of and

implementing tensor2tensor, replacing our earlier 
--------------------------------------------------
h, CA, USA.arXiv:1706.03762v7  [cs.CL]  2 Aug 2023



In [57]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

for chunk in chunks[:5]:
    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = [tokenizer.decode([token]) for token in tokens[:10]]
    print(token_strings)

    print("=" * 50)
    print()

283
[36919, 291, 6300, 63124, 374, 3984, 11, 5195, 22552, 25076]
['Provid', 'ed', ' proper', ' attribution', ' is', ' provided', ',', ' Google', ' hereby', ' grants']

291
[1752, 892, 311, 5542, 13, 5751, 1646, 83691, 220, 1591]
['less', ' time', ' to', ' train', '.', ' Our', ' model', ' achieves', ' ', '28']

83
[95574, 287, 16000, 17, 47211, 11, 25935, 1057, 6931, 2082]
['implement', 'ing', ' tensor', '2', 'tensor', ',', ' replacing', ' our', ' earlier', ' code']



### (2) Hugging Face 토크나이저
- Hugging Face tokenizer 모델이 토큰 수를 기준으로 분할

In [58]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 토크나이저 인코딩
tokens = tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

# 토큰을 출력
print(tokenizer.convert_ids_to_tokens(tokens))

# 디코딩
print(tokenizer.decode(tokens))

/opt/miniconda3/envs/langchain_study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0, 107687, 5, 20451, 54272, 16367, 5, 2]
['<s>', '▁안녕하세요', '.', '▁반', '갑', '습니다', '.', '</s>']
<s> 안녕하세요. 반갑습니다.</s>


In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=0,
)

chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:
    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = tokenizer.convert_ids_to_tokens(tokens[:10])
    print(token_strings)

    print("=" * 50)
    print()

생성된 청크 수: 3
각 청크의 길이: [749, 1094, 1008]

204
[0, 123089, 71, 27798, 99, 179236, 83, 62952, 4, 1815]
['<s>', '▁Provide', 'd', '▁proper', '▁at', 'tribution', '▁is', '▁provided', ',', '▁Google']

248
[0, 51339, 214, 115774, 2843, 37067, 70, 22, 587, 820]
['<s>', '▁perform', 'ing', '▁models', '▁also', '▁connect', '▁the', '▁en', 'co', 'der']

232
[0, 70, 71834, 47, 151575, 13, 903, 6528, 5, 62]
['<s>', '▁the', '▁effort', '▁to', '▁evaluat', 'e', '▁this', '▁idea', '.', '▁A']



## 3.4 Semantic Chunking

In [60]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(),
    breakpoint_threshold_type="gradient",
)

chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:
    # 각 청크를 토큰화

    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 내용을 확인
    print(chunk.page_content[:100])

    print("=" * 50)
    print()

생성된 청크 수: 2
각 청크의 길이: [1736, 1116]

422
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and

260
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and 



## 4. 문서 임베딩(Embedding) 모델
### 4-1 OpenAi

In [ ]:
# embedding 모델
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings = embeddings_model.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings[0])}")
print(document_embeddings[0])

# embed_query 사용
embedded_query = embeddings_model.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

In [ ]:
# 유사도 기반 검색
from langchain_community.utils.math import cosine_similarity
import numpy as np


# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(query, doc_embeddings):
    query_embedding = embeddings_model.embed_query(query)
    # similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in doc_embeddings]
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    most_similar_index = np.argmax(similarities)
    return documents[most_similar_index], similarities[most_similar_index]


# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

## Gradio 챗봇
### (1) invoke 실행
- chat history 기능을 추가

In [61]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

# 답변 생성 모델을 별도로 사용
answer_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

def answer_invoke(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # 현재 메시지에 대해 RAG 체인 실행
    # rag_response = run_route_rag_chain(message)

    # 답변 생성 모델에게 현재 메시지에 대한 답변 요청
    # final_answer = answer_llm.invoke(
    #     history_langchain_format[:-1]+[AIMessage(content=rag_response)]+[HumanMessage(content=message)]
    # )
    # return final_answer.content

NameError: name 'ChatOpenAI' is not defined